In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import zipfile
import os

# Chemin vers le fichier ZIP (MODIFIE CE CHEMIN !)
zip_path = "/content/drive/MyDrive/archive ocr 4.zip"

# Chemin d'extraction
extract_path = "/content/dataset"

# Décompression
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Vérifier les fichiers extraits
print("Décompression terminée !")
print(os.listdir(extract_path))  # Affiche le contenu du dossier


Décompression terminée !
['CNN letter Dataset']


In [3]:
import os

dataset_path = "/content/dataset/CNN letter Dataset"  # Mets le bon chemin si nécessaire

# Lister les fichiers et dossiers à l'intérieur
print(os.listdir(dataset_path))


['E', 'W', 'L', '7', 'N', 'K', 'R', 'U', '5', 'V', 'A', '0', '8', '4', 'H', 'J', 'G', '6', 'D', 'X', '3', 'T', 'Z', '2', 'F', 'C', 'P', 'Q', 'B', '1', 'Y', 'M', 'I', '9', 'S']


In [4]:
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):  # Vérifier que c'est bien un dossier
        print(f"Classe {label} : {len(os.listdir(label_path))} images")


Classe E : 1010 images
Classe W : 1010 images
Classe L : 1010 images
Classe 7 : 1030 images
Classe N : 1020 images
Classe K : 1010 images
Classe R : 1020 images
Classe U : 1010 images
Classe 5 : 1030 images
Classe V : 1030 images
Classe A : 1010 images
Classe 0 : 1030 images
Classe 8 : 1030 images
Classe 4 : 1030 images
Classe H : 1020 images
Classe J : 1030 images
Classe G : 1020 images
Classe 6 : 1030 images
Classe D : 1010 images
Classe X : 1010 images
Classe 3 : 1030 images
Classe T : 1020 images
Classe Z : 810 images
Classe 2 : 1030 images
Classe F : 1020 images
Classe C : 1020 images
Classe P : 1010 images
Classe Q : 1010 images
Classe B : 1030 images
Classe 1 : 1030 images
Classe Y : 1010 images
Classe M : 1020 images
Classe I : 1010 images
Classe 9 : 1030 images
Classe S : 1020 images


In [5]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

data = []
labels = []

for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)

    if os.path.isdir(label_path):
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)

            # Charger l'image en niveaux de gris
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is not None:
                img = cv2.resize(img, (75, 100))  # Uniformiser la taille
                img = img_to_array(img) / 255.0  # Normalisation entre 0 et 1

                data.append(img)
                labels.append(label)  # Ajouter le label associé

# Convertir en tableaux numpy
data = np.array(data, dtype="float32")
labels = np.array(labels)

print(f"Nombre total d'images chargées : {len(data)}")
print(f"Exemple de label : {labels[0]}")


Nombre total d'images chargées : 35500
Exemple de label : E


In [6]:
from sklearn.preprocessing import LabelEncoder

# Encoder les labels en entiers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Vérification
print("Exemple de label encodé :", labels_encoded[0])
print("Mapping des classes :", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Exemple de label encodé : 14
Mapping des classes : {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'J': 19, 'K': 20, 'L': 21, 'M': 22, 'N': 23, 'P': 24, 'Q': 25, 'R': 26, 'S': 27, 'T': 28, 'U': 29, 'V': 30, 'W': 31, 'X': 32, 'Y': 33, 'Z': 34}


In [7]:
from tensorflow.keras.utils import to_categorical

# Conversion en One-Hot Encoding
y_one_hot = to_categorical(labels_encoded, num_classes=35)

# Vérification
print("Exemple de y_one_hot :", y_one_hot[0])
print("Shape de y_one_hot :", y_one_hot.shape)


Exemple de y_one_hot : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Shape de y_one_hot : (35500, 35)


In [8]:
import numpy as np

# Conversion des images en tableau NumPy
X = np.array(data, dtype="float32")

# Vérification
print(f"Shape de X : {X.shape}")
print(f"Valeur min/max avant normalisation : {X.min()} / {X.max()}")


Shape de X : (35500, 100, 75, 1)
Valeur min/max avant normalisation : 0.0 / 1.0


In [9]:
from sklearn.model_selection import train_test_split

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Vérification des dimensions
print(f"Shape de X_train : {X_train.shape}")
print(f"Shape de X_test : {X_test.shape}")
print(f"Shape de y_train : {y_train.shape}")
print(f"Shape de y_test : {y_test.shape}")


Shape de X_train : (28400, 100, 75, 1)
Shape de X_test : (7100, 100, 75, 1)
Shape de y_train : (28400, 35)
Shape de y_test : (7100, 35)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Création du modèle CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100, 75, 1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(35, activation='softmax')  # 35 classes pour la classification
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 73, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 49, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 47, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 21, 15, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 7, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8960)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,147,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 35)                  │           4,515 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,244,195 (4.75 MB)

 Trainable params: 1,244,195 (4.75 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Entraînement du modèle
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)


Epoch 1/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 368s 412ms/step - accuracy: 0.4806 - loss: 1.8218 - val_accuracy: 0.9776 - val_loss: 0.0827
Epoch 2/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 346s 389ms/step - accuracy: 0.8930 - loss: 0.3230 - val_accuracy: 0.9925 - val_loss: 0.0338
Epoch 3/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 346s 390ms/step - accuracy: 0.9360 - loss: 0.2019 - val_accuracy: 0.9939 - val_loss: 0.0242
Epoch 4/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 403s 413ms/step - accuracy: 0.9494 - loss: 0.1500 - val_accuracy: 0.9966 - val_loss: 0.0161
Epoch 5/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 358s 387ms/step - accuracy: 0.9614 - loss: 0.1196 - val_accuracy: 0.9983 - val_loss: 0.0080
Epoch 6/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 346s 389ms/step - accuracy: 0.9671 - loss: 0.0975 - val_accuracy: 0.9985 - val_loss: 0.0064
Epoch 7/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 378s 384ms/step - accuracy: 0.9725 - loss: 0.0827 - val_accuracy: 0.9982 - val_loss: 0.0057
Epoch 8/20
888/888 ━━━━━━━━━━━━━━━━━━━━ 366s 412ms/step - accuracy: 0.9747 -

In [12]:
# Chemin où sauvegarder le modèle dans Google Drive
model_path = "/content/drive/My Drive/modele_ocr_plate.h5"

# Sauvegarde du modèle
model.save(model_path)
print(f"Modèle sauvegardé à : {model_path}")

Modèle sauvegardé à : /content/drive/My Drive/modele_ocr_plate.h5


In [13]:
from tensorflow.keras.models import load_model

# Charger le modèle depuis Google Drive
model = load_model(model_path)
print("Modèle chargé avec succès !")


Modèle chargé avec succès !


In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Modèle compilé avec succès !")


Modèle compilé avec succès !


In [15]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy sur le test set : {accuracy * 100:.2f}%")


222/222 ━━━━━━━━━━━━━━━━━━━━ 22s 95ms/step - accuracy: 0.9998 - loss: 7.2227e-04
Accuracy sur le test set : 99.94%


In [16]:
import numpy as np

# Prendre une image du dataset de test
image_test = X_test[0]  # Prendre la première image
image_test = np.expand_dims(image_test, axis=0)  # Ajouter une dimension batch

# Prédiction
prediction = model.predict(image_test)
predicted_class = np.argmax(prediction)  # Trouver l'indice de la classe prédite

# Récupérer le label correspondant
class_labels = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
                10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I',
                19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'P', 25: 'Q', 26: 'R', 27: 'S',
                28: 'T', 29: 'U', 30: 'V', 31: 'W', 32: 'X', 33: 'Y', 34: 'Z'}

predicted_label = class_labels[predicted_class]

print(f"Classe prédite : {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Classe prédite : P


In [19]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall